# Types of ML Systems

# Based on Supervision

![Meaning of supervise](images/supervise_meaning.png)

## Supervised Learning

The training data is labeled
<!-- ![Supervised Learning](images/supervised_learning.png) -->
<img src="images/supervised_learning.png" style="width:500px"><br>

### Two types:
- **Classification**: Classifies into discrete classes
- **Regression**: Predicts numerical value

**Note**: Some *regression* algorithms (e.g. **Logistic Regression**) is used for classification

### Some supervised algorithms, we will read about:
- k Nearest Neighbour
- Linear Regression
- Logistic Regression
- SVM
- Decision Tree & Random Forest

## Unsupervised Learning

Training data is ***not*** labeled

### Four Types:
- Clustering
- Anomaly detection
- Visualization and dimensionality reduction
- Association rule learning

## Semi-supervised Learning

Training data is ***partially*** labeled (usually, mostly unlabled) <br>
Because, labeling data is quite expensive

### Example: Google Photos
- Recognizes similar faces and clusters them together (ideally)
- Once the user labels the clusters, it can name everyone in every photo

## Reinforcement Learning

Very different from the other three.

The agent (learning system) performs actions and gets reward (or penalties) in return. <br>
The goal is to get most rewards. <br>
The rewards/penalties guides the agent to choose the best strategy (called *policy*) <br>
<img src="images/reinforcement_learning.png" style="width:500px;">